In [1]:
import matplotlib.pyplot as plt
from itertools import combinations
from random import random,randint
import numpy as np
from functools import reduce
from scipy.optimize import minimize
from scipy.linalg import expm,sinm,cosm

import time
import math
from   matplotlib import cm
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute
from qiskit.quantum_info.analysis import average_data
from qiskit import *
from qiskit.quantum_info.operators import Operator
from qiskit.aqua.utils import tensorproduct
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from qiskit.providers.ibmq      import least_busy
from qiskit.tools.monitor       import job_monitor
from qiskit.visualization import plot_histogram
import picos as pic
import warnings
from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import CplexOptimizer
import scipy.linalg as la
from itertools import permutations, combinations


In [2]:
def num_1_pos(d):
    get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
    num_1 = get_indexes(1,d)
    return num_1,len(num_1)

def num_0_pos(d):
    get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
    num_0 = get_indexes(0,d)
    return num_0,len(num_0)

def column(matrix, i):
    return [row[i] for row in matrix]

#Construct initial state 
def init_s(n):
    x = pow(2,n)
    s = 1/np.sqrt(x) * np.ones(x)
    s = s.reshape(x, 1)
    return s

def bin2dec(string_num):
    return str(int(string_num, 2))

def row_binary(v):
    for i in range(len(v)):
        if v[i]%2 == 0:
            v[i] = 0
        else:
            v[i] = 1 
    return v

def converter(d):
    P = 0
    for i in range(len(d)):
        P+=d[i]*(2**(len(d)-i-1))
    return P
                 
#Define some pauli operators
PauliX = [[0,1],[1,0]]   
PauliZ = [[1,0],[0,-1]]
PauliI = np.eye(2)

#Define Cjk and B
def Cjk_generator(d):
    n = len(d)
    l = list(range(0,n))    
    for i in range(n):
        l[i] = PauliI
    s = num_1_pos(d)[0]
    for j in range(len(s)):
        l[s[j]] = PauliZ
    Cjk = l[0]
    for i in range(n-1):
        Cjk = np.kron(Cjk,l[i+1])
    return Cjk

def B_generator(n):
    l = list(range(0,n))
    b = list(range(0,n))
    
    for a in range(n):
        for i in range(n):
            l[i] = PauliI
        l[a]=PauliX
        temp = l[0]
        for i in range(n-1):
            temp = np.kron(temp,l[i+1])
        b[a] = temp
    B = sum(b[a] for a in range(n))
    return B

def basis(n):
    nums = np.array([a for a in range(2**(n))]) 
    bin_nums = (((nums.reshape(-1,1) & (2**np.arange(n)))) != 0).astype(int)
    a = bin_nums[:,::-1]
    return a


def matrix_binary(A):
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i][j]%2 == 0:
                A[i][j] = 0
            else:
                A[i][j] = 1
    return A

def error(n,err):
    e = np.zeros(n)
    a = np.random.randint(n,size=err)
    for i in range(len(a)):
        e[a[i]] = 1 
    return e
    




# Oringinal Generator matrix without error

In [3]:
G_oringin = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
                      [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1],
                      [0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1],
                      [0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1],
                      [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]])
G_oringinT = G_oringin.T

In [6]:


n = G_oringin.shape[0]
u=0

for k in range(2**n):
    print("$$$$$$$$$$$$$$$$$$$")
    message = basis(n)[k]
    y = row_binary(message.dot(G_oringin))
    print("y = ",y)
    C = sum((1-2*y[a])*Cjk_generator(G_oringinT[a]) for a in range(G_oringinT.shape[0]))
    B = B_generator(G_oringinT.shape[1])
    s = init_s(n)
    s_d = (s).T
    print("the number of basis",k+1)

    #run the program
    start_time = time.time()
    step_size   = 0.05
    result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
    a_gamma         = np.arange(0, np.pi, step_size)
    a_beta          = np.arange(0, np.pi, step_size)
    #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
    f_p_max = 0
    f_p_max_gamma = 0
    f_p_max_beta = 0
    f_psum = 0

    for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
        for j in range(0, int(3.14/step_size)+1):       # j for a_beta

            #Construct U_C
            U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
            #Construct U_B
            U_B = expm(-1j*a_beta[j]*B)


            #Construct expection of each subgraph
            state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
            state_d = (state.conjugate()).T # bra


            f_p = (state_d.dot(C)).dot(state)
            f_psum += f_p


            if f_p.real > f_p_max:
                f_p_max = f_p.real
                f_p_max_gamma = a_gamma[i]
                f_p_max_beta = a_beta[j]

            result_array[i][j] = f_p.real
    print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
    print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
    print("\n Optimal gamma : ",f_p_max_gamma)
    print("\n Optimal beta : ",f_p_max_beta)


    end_time = time.time()
    #print("\n Runningtime : ",end_time - start_time)
    c = converter(message)
    r = np.ones(2**len(message))
    #Construct U_C
    U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
    #Construct U_B
    U_B = expm(-1j*f_p_max_beta*B)


    #Construct expection of each subgraph
    state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
    prob=[]
    for i in range(len(state)):
        p = (state[i].real)**2 + (state[i].imag)**2
        np.array(prob.append(float(p)))      
    e = np.max(prob)
    print(e)
    print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
    if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
        print("CANNOT correct！！！！！！")
    else:
        print("Good Job!!!!!!")
        u+=1

print("---------------")
print('correct rate: ',u/(2**n)*100,'%')



$$$$$$$$$$$$$$$$$$$
y =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
the number of basis 1

 Average expectation using QAOA with fine grid :  [[1.2705046e-06]]

 Maximum expectation using QAOA with fine grid :  [[1.21517014]]

 Optimal gamma :  0.75

 Optimal beta :  2.9000000000000004
2.54323232981552
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
the number of basis 2

 Average expectation using QAOA with fine grid :  [[1.2705046e-06]]

 Maximum expectation using QAOA with fine grid :  [[1.21517014]]

 Optimal gamma :  0.75

 Optimal beta :  2.9000000000000004
2.543232329815519
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
the number of basis 3

 Average expectation using QAOA with fine grid :  [[1.2705046e-06]]

 Maximum expectation using QAOA with fine grid :  [[1.21517014]]

 Optimal gamma :  0.75

 Optimal beta :  2.9000000000000004
2.54323232981552
most likey basis:   3
Good Job!!!!!!
$$$$$$


 Average expectation using QAOA with fine grid :  [[1.2705046e-06]]

 Maximum expectation using QAOA with fine grid :  [[1.21517014]]

 Optimal gamma :  0.75

 Optimal beta :  2.9000000000000004
2.54323232981552
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [1 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1]
the number of basis 27

 Average expectation using QAOA with fine grid :  [[1.2705046e-06]]

 Maximum expectation using QAOA with fine grid :  [[1.21517014]]

 Optimal gamma :  0.75

 Optimal beta :  2.9000000000000004
2.5432323298155213
most likey basis:   27
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0]
the number of basis 28

 Average expectation using QAOA with fine grid :  [[1.2705046e-06]]

 Maximum expectation using QAOA with fine grid :  [[1.21517014]]

 Optimal gamma :  0.75

 Optimal beta :  2.9000000000000004
2.543232329815519
most likey basis:   28
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
y =  [1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1]
the number of basis 2

# Oringinal with one bit error

# Generate one bit error pattern

In [4]:
w1 = []
for i in range(G_oringin.shape[1]):
    o = np.zeros(G_oringin.shape[1])
    o[i]=1
    w1.append(o)      

In [5]:
G_oringinT = G_oringin.T
cor_rate=[]
n = G_oringin.shape[0]

for m in range(len(w1)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w1[m] 
        print("w = ",w)
        y = row_binary(message.dot(G_oringin)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(G_oringinT[a]) for a in range(G_oringinT.shape[0]))
        B = B_generator(G_oringinT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print(len(np.where(prob==np.max(prob))[0]))
        print("most likey basis:  ", int(np.where(prob==np.max(prob))[0][0]+1))
        if len(np.where(prob==np.max(prob))[0]) == 1:
            if int(k+1) != int(np.where(prob==np.max(prob))[0][0]+1):
                print("CANNOT correct！！！！！！")
            else:
                print("Good Job!!!!!!")
                u+=1
        else:
            print("CANNOT correct！！！！！！")
    print("w: " + str(w))
    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)


$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
1
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 2
1
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
the number of basis 3
1
most likey basis:   3
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 4
1
most likey basis:   4
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
the number of basis 5
1
most likey basis:   5
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 0

1
most likey basis:   12
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
the number of basis 12
1
most likey basis:   11
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
the number of basis 13
1
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.]
the number of basis 14
1
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
the number of basis 15
1
most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0.]
the number of basis 16
1
mos

1
most likey basis:   18
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
the number of basis 21
1
most likey basis:   23
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1.]
the number of basis 22
1
most likey basis:   24
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1.]
the number of basis 23
1
most likey basis:   21
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0.]
the number of basis 24
1
most likey basis:   22
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 25
1
mos

1
most likey basis:   6
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0.]
the number of basis 31
1
most likey basis:   11
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1.]
the number of basis 32
1
most likey basis:   12
CANNOT correct！！！！！！
w: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---------------
correct rate:  0.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
1
most likey basis:   5
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 2
1
most likey basis:   6
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

1
most likey basis:   31
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1.]
the number of basis 8
1
most likey basis:   22
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
the number of basis 9
1
most likey basis:   17
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0.]
the number of basis 10
1
most likey basis:   18
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0.]
the number of basis 11
2
most likey basis:   19
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
the number of basis 12
2
most likey basis:   20
$$$

...

2
most likey basis:   26
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1.]
the number of basis 15
1
most likey basis:   32
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
the number of basis 16
2
most likey basis:   28
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
the number of basis 17
1
most likey basis:   2
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
the number of basis 18
1
most likey basis:   6
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0.]
the number of basis 19
1
most likey basis:   4
CANNOT correct！！！！！！
$$$$

1
most likey basis:   4
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.]
the number of basis 25
1
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1.]
the number of basis 26
1
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.]
the number of basis 27
1
most likey basis:   15
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0.]
the number of basis 28
1
most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
the number of basis 29
1
most

1
most likey basis:   19
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1.]
the number of basis 2
1
most likey basis:   20
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1.]
the number of basis 3
1
most likey basis:   17
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0.]
the number of basis 4
1
most likey basis:   18
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1.]
the number of basis 5
1
most likey basis:   23
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0.]
the number of basis 6
1
most lik

1
most likey basis:   25
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0.]
the number of basis 11
1
most likey basis:   28
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1.]
the number of basis 12
1
most likey basis:   27
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0.]
the number of basis 13
1
most likey basis:   30
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1.]
the number of basis 14
1
most likey basis:   29
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
the number of basis 15
1
mos

1
most likey basis:   20
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1.]
the number of basis 21
1
most likey basis:   21
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.]
the number of basis 22
1
most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]
the number of basis 23
1
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1.]
the number of basis 24
1
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
the number of basis 25
1
most likey basis:   25
Good Job!!

# Oringinal with two bit error

In [12]:
w2 = []
comb = combinations([a for a in range(G_oringin.shape[1])], 2) 
comb = list(comb)
for a in range(len(comb)):
    o = np.zeros(G_oringin.shape[1])
    j = comb[a][0]
    k = comb[a][1]
    o[j]=1
    o[k]=1
    w2.append(o)

print(w2)
   
    


[array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [9]:

G_oringinT = G_oringin.T
cor_rate=[]
n = G_oringin.shape[0]

for m in range(len(w2)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w2[m] 
        print("w = ",w)
        y = row_binary(message.dot(G_oringin)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(G_oringinT[a]) for a in range(G_oringinT.shape[0]))
        B = B_generator(G_oringinT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0][0])+1)
        if len(np.where(prob==np.max(prob))[0]) == 1:
            if int(k+1) != int(np.where(prob==np.max(prob))[0][0]+1):
                print("CANNOT correct！！！！！！")
            else:
                print("Good Job!!!!!!")
                u+=1
        else:
            print("CANNOT correct！！！！！！")
    print("w: " + str(w))
    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)




$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 2
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
the number of basis 3
most likey basis:   3
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 4
most likey basis:   4
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
the number of basis 5
most likey basis:   5
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.

most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0.]
the number of basis 16
most likey basis:   16
Good Job!!!!!!
$$$$$$$

most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1.]
the number of basis 23
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0.]
the number of basis 24
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 25
most likey basis:   25
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 26
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1.]
the number of basis 27
most likey basis:   27
Good Job!!!!!!
$$$$$$$

most likey basis:   1
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 2
most likey basis:   2
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
the number of basis 3
most likey basis:   3
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 4
most likey basis:   4
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
the number of basis 5
most likey basis:   5
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0.]
the number of basis 6
most likey basis:   6
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$

most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0.]
the number of basis 16
most likey basis:   16
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
the number of basis 17
most likey basis:   17
Good Job!!!!!!
$$$$$$$

...

most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
y =  [0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0.]
the number of basis 16
most likey basis:   16
Good Job!!!!!!
$$$$$$$

most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1.]
the number of basis 23
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0.]
the number of basis 24
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0.]
the number of basis 25
most likey basis:   25
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1.]
the number of basis 26
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
y =  [1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1.]
the number of basis 27
most likey basis:   27
Good Job!!!!!!
$$$$$$$

most likey basis:   29
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1.]
the number of basis 2
most likey basis:   30
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1.]
the number of basis 3
most likey basis:   31
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
the number of basis 4
most likey basis:   32
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
the number of basis 5
most likey basis:   25
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
y =  [0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0.]
the number of basis 6
most likey basis:   

most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
y =  [0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.]
the number of basis 16
most likey basis:   16
Good Job!!!!!!
$$$$$$$

most likey basis:   22
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
y =  [1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
the number of basis 23
most likey basis:   23
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
y =  [1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.]
the number of basis 24
most likey basis:   24
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
y =  [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
the number of basis 25
most likey basis:   25
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
y =  [1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0.]
the number of basis 26
most likey basis:   26
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
y =  [1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
the number of basis 27
most likey basis:   27
Good Job!!!!!!
$$$$$$$

In [11]:
u=0
for i in range(len(cor_rate)):
    if cor_rate[i]==100:
        u+=1
    else:
        continue;
print(u/len(cor_rate))

0.6916666666666667


# There bit error

In [15]:
w3 = []
comb = combinations([a for a in range(G_oringin.shape[1])], 3) 
comb = list(comb)
for a in range(len(comb)):
    o = np.zeros(G_oringin.shape[1])
    j = comb[a][0]
    k = comb[a][1]
    l = comb[a][2]
    o[j]=1
    o[k]=1
    o[l]=1
    w3.append(o)

In [16]:

G_oringinT = G_oringin.T
cor_rate=[]
n = G_oringin.shape[0]

for m in range(len(w2)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w3[m] 
        print("w = ",w)
        y = row_binary(message.dot(G_oringin)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(G_oringinT[a]) for a in range(G_oringinT.shape[0]))
        B = B_generator(G_oringinT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0][0])+1)
        if len(np.where(prob==np.max(prob))[0]) == 1:
            if int(k+1) != int(np.where(prob==np.max(prob))[0][0]+1):
                print("CANNOT correct！！！！！！")
            else:
                print("Good Job!!!!!!")
                u+=1
        else:
            print("CANNOT correct！！！！！！")
    print("w: " + str(w))
    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)



$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   29
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 2
most likey basis:   30
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
the number of basis 3
most likey basis:   31
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0.]
the number of basis 4
most likey basis:   32
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
the number of basis 5
most likey basis:   25
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$


most likey basis:   10
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0.]
the number of basis 11
most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
the number of basis 15
most likey basis:   15
Good Job!!!!!!
$$$$$$$

most likey basis:   10
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
the number of basis 21
most likey basis:   15
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1.]
the number of basis 22
most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1.]
the number of basis 23
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0.]
the number of basis 24
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 25
most likey basi

most likey basis:   31
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1.]
the number of basis 32
most likey basis:   32
Good Job!!!!!!
w: [1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---------------
correct rate:  100.0 %
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   25
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 2
most likey basis:   26
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
the number of basis 3
most likey basis:   27
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0

most likey basis:   9
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
the number of basis 10
most likey basis:   10
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0.]
the number of basis 11
most likey basis:   11
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1.]
the number of basis 12
most likey basis:   12
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
the number of basis 13
most likey basis:   13
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1.]
the number of basis 14
most likey basis:   14
Good Job!!!!!!
$$$$$$$$

...

most likey basis:   32
Good Job!!!!!!
w: [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
---------------
correct rate:  100.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
the number of basis 1
most likey basis:   29
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1.]
the number of basis 2
most likey basis:   30
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.]
the number of basis 3
most likey basis:   31
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0.]
the number of basis 4
most likey basis:   32
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
y =  [0. 1.

most likey basis:   21
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0.]
the number of basis 10
most likey basis:   22
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0.]
the number of basis 11
most likey basis:   23
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1.]
the number of basis 12
most likey basis:   24
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0.]
the number of basis 13
most likey basis:   17
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y =  [0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.]
the number of basis 14
most likey basi

most likey basis:   15
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1.]
the number of basis 20
most likey basis:   16
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0.]
the number of basis 21
most likey basis:   9
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1.]
the number of basis 22
most likey basis:   10
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
the number of basis 23
most likey basis:   11
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
y =  [1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
the number of basis 24
most likey basis

most likey basis:   30
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1.]
the number of basis 31
most likey basis:   31
Good Job!!!!!!
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
y =  [1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
the number of basis 32
most likey basis:   32
Good Job!!!!!!
w: [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
---------------
correct rate:  100.0 %
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
the number of basis 1
most likey basis:   14
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
the number of basis 2
most likey basis:   13
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 1. 

most likey basis:   31
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1.]
the number of basis 8
most likey basis:   32
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
the number of basis 9
most likey basis:   17
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0.]
the number of basis 10
most likey basis:   18
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0.]
the number of basis 11
most likey basis:   19
CANNOT correct！！！！！！
$$$$$$$$$$$$$$$$$$$
w =  [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y =  [0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
the number of basis 12
most likey basis:

In [17]:
u=0
for i in range(len(cor_rate)):
    if cor_rate[i]==100:
        u+=1
    else:
        continue;
print(u/len(cor_rate))

0.625


# Apply a full rank matrix on G

In [ ]:
G_oringin = np.array([[1,0,0,1,0,1],
                      [0,1,0,1,1,1],
                      [0,0,1,1,1,0]])
P = np.array([[1,1,0],
              [1,0,1],
              [0,1,0]])
print(np.linalg.matrix_rank(P))
G = matrix_binary(P.dot(G_oringin))
print(G)

# Without error

In [ ]:
GT = G.T

n = G.shape[0]
u=0

for k in range(2**n):
    print("$$$$$$$$$$$$$$$$$$$")
    message = basis(n)[k]
    y = row_binary(message.dot(G))
    print("y = ",y)
    C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
    B = B_generator(GT.shape[1])
    s = init_s(n)
    s_d = (s).T
    print("the number of basis",k+1)

    #run the program
    start_time = time.time()
    step_size   = 0.05
    result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
    a_gamma         = np.arange(0, np.pi, step_size)
    a_beta          = np.arange(0, np.pi, step_size)
    print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
    f_p_max = 0
    f_p_max_gamma = 0
    f_p_max_beta = 0
    f_psum = 0

    for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
        for j in range(0, int(3.14/step_size)+1):       # j for a_beta

            #Construct U_C
            U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
            #Construct U_B
            U_B = expm(-1j*a_beta[j]*B)


            #Construct expection of each subgraph
            state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
            state_d = (state.conjugate()).T # bra


            f_p = (state_d.dot(C)).dot(state)
            f_psum += f_p


            if f_p.real > f_p_max:
                f_p_max = f_p.real
                f_p_max_gamma = a_gamma[i]
                f_p_max_beta = a_beta[j]

            result_array[i][j] = f_p.real
    print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
    print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
    print("\n Optimal gamma : ",f_p_max_gamma)
    print("\n Optimal beta : ",f_p_max_beta)


    end_time = time.time()
    print("\n Runningtime : ",end_time - start_time)
    c = converter(message)
    r = np.ones(2**len(message))
    print(r)
    #Construct U_C
    U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
    #Construct U_B
    U_B = expm(-1j*f_p_max_beta*B)


    #Construct expection of each subgraph
    state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
    print(state)
    prob=[]
    for i in range(len(state)):
        p = (state[i].real)**2 + (state[i].imag)**2
        np.array(prob.append(float(p)))
    print(prob)      
    e = np.max(prob)
    print(e)
    print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
    if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
        print("CANNOT correct！！！！！！")
    else:
        print("Good Job!!!!!!")
        u+=1

print("---------------")
print('correct rate: ',u/(2**n)*100,'%')

# One bit error decode

In [ ]:
GT = G.T

n = G.shape[0]
cor_rate=[]
for m in range(len(w1)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w1[m] 
        print("w = ",w)
        y = row_binary(message.dot(G)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
        B = B_generator(GT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
        if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
            print("CANNOT correct！！！！！！")
        else:
            print("Good Job!!!!!!")
            u+=1

    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)





# Two bits error decode

In [ ]:
GT = G.T

n = G.shape[0]
u=0
cor_rate=[]
for m in range(len(w2)):
    u=0
    for k in range(2**n):
        print("$$$$$$$$$$$$$$$$$$$")
        message = basis(n)[k]
        w = w2[m] 
        print("w = ",w)
        y = row_binary(message.dot(G)+w)
        print("y = ",y)
        C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
        B = B_generator(GT.shape[1])
        s = init_s(n)
        s_d = (s).T
        print("the number of basis",k+1)

        #run the program
        start_time = time.time()
        step_size   = 0.05
        result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
        a_gamma         = np.arange(0, np.pi, step_size)
        a_beta          = np.arange(0, np.pi, step_size)
        #print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
        f_p_max = 0
        f_p_max_gamma = 0
        f_p_max_beta = 0
        f_psum = 0

        for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
            for j in range(0, int(3.14/step_size)+1):       # j for a_beta

                #Construct U_C
                U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
                #Construct U_B
                U_B = expm(-1j*a_beta[j]*B)


                #Construct expection of each subgraph
                state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
                state_d = (state.conjugate()).T # bra


                f_p = (state_d.dot(C)).dot(state)
                f_psum += f_p


                if f_p.real > f_p_max:
                    f_p_max = f_p.real
                    f_p_max_gamma = a_gamma[i]
                    f_p_max_beta = a_beta[j]

                result_array[i][j] = f_p.real
#         print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
#         print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
#         print("\n Optimal gamma : ",f_p_max_gamma)
#         print("\n Optimal beta : ",f_p_max_beta)


        end_time = time.time()
        #print("\n Runningtime : ",end_time - start_time)
        c = converter(message)
        r = np.ones(2**len(message))

        #Construct U_C
        U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
        #Construct U_B
        U_B = expm(-1j*f_p_max_beta*B)


        #Construct expection of each subgraph
        state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
        prob=[]
        for i in range(len(state)):
            p = (state[i].real)**2 + (state[i].imag)**2
            np.array(prob.append(float(p)))      
        e = np.max(prob)
        print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
        if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
            print("CANNOT correct！！！！！！")
        else:
            print("Good Job!!!!!!")
            u+=1

    print("---------------")
    print('correct rate: ',u/(2**n)*100,'%')

    cor_rate.append(u/(2**n)*100)
print('----------------------')
print(cor_rate)

# Three bits error decode

In [ ]:
GT = G.T

n = G.shape[0]
u=0

for k in range(2**n):
    print("$$$$$$$$$$$$$$$$$$$")
    message = basis(n)[k]
    w = error(G.shape[1],3) 
    print("w = ",w)
    y = row_binary(message.dot(G)+w)
    print("y = ",y)
    C = sum((1-2*y[a])*Cjk_generator(GT[a]) for a in range(GT.shape[0]))
    B = B_generator(GT.shape[1])
    s = init_s(n)
    s_d = (s).T
    print("the number of basis",k+1)

    #run the program
    start_time = time.time()
    step_size   = 0.05
    result_array = np.zeros(shape=(int(3.14/step_size)+1, int(3.14/step_size)+1))
    a_gamma         = np.arange(0, np.pi, step_size)
    a_beta          = np.arange(0, np.pi, step_size)
    print("Numbers of iteration: ",(pow((int(3.14/step_size)+1),2)))
    f_p_max = 0
    f_p_max_gamma = 0
    f_p_max_beta = 0
    f_psum = 0

    for i in range(0, int(3.14/step_size)+1):           # i for a_gamma
        for j in range(0, int(3.14/step_size)+1):       # j for a_beta

            #Construct U_C
            U_C = np.asmatrix(expm(-1j*2*a_gamma[i]*C))
            #Construct U_B
            U_B = expm(-1j*a_beta[j]*B)


            #Construct expection of each subgraph
            state = (U_B.dot(U_C)).dot(s).reshape(pow(2,n),1) # ket
            state_d = (state.conjugate()).T # bra


            f_p = (state_d.dot(C)).dot(state)
            f_psum += f_p


            if f_p.real > f_p_max:
                f_p_max = f_p.real
                f_p_max_gamma = a_gamma[i]
                f_p_max_beta = a_beta[j]

            result_array[i][j] = f_p.real
    print("\n Average expectation using QAOA with fine grid : ",(f_psum/pow(len(a_gamma),2)).real)       
    print("\n Maximum expectation using QAOA with fine grid : ",f_p_max)
    print("\n Optimal gamma : ",f_p_max_gamma)
    print("\n Optimal beta : ",f_p_max_beta)


    end_time = time.time()
    print("\n Runningtime : ",end_time - start_time)
    c = converter(message)
    r = np.ones(2**len(message))
    print(r)
    #Construct U_C
    U_C = np.asmatrix(expm(-1j*2*f_p_max_gamma*C))
    #Construct U_B
    U_B = expm(-1j*f_p_max_beta*B)


    #Construct expection of each subgraph
    state = (U_B.dot(U_C)).dot(r).reshape(pow(2,n),1) # ket
    print(state)
    prob=[]
    for i in range(len(state)):
        p = (state[i].real)**2 + (state[i].imag)**2
        np.array(prob.append(float(p)))
    print(prob)      
    e = np.max(prob)
    print(e)
    print("most likey basis:  ",int(np.where(prob==np.max(prob))[0])+1)
    if int(k+1) != int(np.where(prob==np.max(prob))[0]+1):
        print("CANNOT correct！！！！！！")
    else:
        print("Good Job!!!!!!")
        u+=1
print("---------------")
print('correct rate: ',u/16*100,'%')